# Named Entity Recognition using LSTM

### 1. Preparing vocabulary for Word Embeddings

In [1]:
import pandas as pd
import numpy as np
import nltk

In [2]:
df = pd.read_csv('./data/ner_dataset.csv')

In [3]:
words = df['Word'].str.lower()
words = words.drop_duplicates().values
print(words)
print(words.shape)

['thousands' 'of' 'demonstrators' ..., 'indicative' '3700' 'bermel']
(31804,)


In [4]:
word_indices = {}

In [5]:
def map_word_indices(words):
    for i in range(len(words)):
        word_indices[words[i]] = i

In [6]:
map_word_indices(words)

In [7]:
def get_index_matrix(s):
    id_matrix = []
    w = nltk.word_tokenize(s)
    w = [i.lower() for i in w]
    for i in w:
        if i in words:
            id_matrix.append(word_indices[i])
        else :
            id_matrix.append(31805) #Unknown token
    return id_matrix

In [9]:
sentence = "My name is Edward."
print(get_index_matrix(sentence))

[1396, 2960, 75, 10085, 21]


Generating this ID matrix will be useful for training the embedding layer.

### 2. Preparing Data-set

In [10]:
df.head()

,Sentence,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [11]:
tags = df['Tag'].drop_duplicates().values
print(tags)
print("Number of classes : " + str(len(tags)))

['O' 'B-geo' 'B-gpe' 'B-per' 'I-geo' 'B-org' 'I-org' 'B-tim' 'B-art'
 'I-art' 'I-per' 'I-gpe' 'I-tim' 'B-nat' 'B-eve' 'I-eve' 'I-nat']
Number of classes : 17


In [12]:
one_hot_tags = pd.get_dummies(df['Tag']).values

In [13]:
x_data = []
y_data = []
temp_y = []
s = ''
for index,rows in df.iterrows():
    s = s + rows['Word'].lower() + ' '
    temp_y.append(one_hot_tags[index])
    if rows['Word'] == ".":
        x_data.append(get_index_matrix(s))
        y_data.append(temp_y)
        temp_y = []
        s = ''
        print('Sentences formatted : ' + str(len(x_data)))

Sentences formatted : 1
Sentences formatted : 2
Sentences formatted : 3
Sentences formatted : 4
Sentences formatted : 5
Sentences formatted : 6
Sentences formatted : 7
Sentences formatted : 8
Sentences formatted : 9
Sentences formatted : 10
Sentences formatted : 11
Sentences formatted : 12
Sentences formatted : 13
Sentences formatted : 14
Sentences formatted : 15
Sentences formatted : 16
Sentences formatted : 17
Sentences formatted : 18
Sentences formatted : 19
Sentences formatted : 20
Sentences formatted : 21
Sentences formatted : 22
Sentences formatted : 23
Sentences formatted : 24
Sentences formatted : 25
Sentences formatted : 26
Sentences formatted : 27
Sentences formatted : 28
Sentences formatted : 29
Sentences formatted : 30
Sentences formatted : 31
Sentences formatted : 32
Sentences formatted : 33
Sentences formatted : 34
Sentences formatted : 35
Sentences formatted : 36
Sentences formatted : 37
Sentences formatted : 38
Sentences formatted : 39
Sentences formatted : 40
Sentences

Sentences formatted : 368
Sentences formatted : 369
Sentences formatted : 370
Sentences formatted : 371
Sentences formatted : 372
Sentences formatted : 373
Sentences formatted : 374
Sentences formatted : 375
Sentences formatted : 376
Sentences formatted : 377
Sentences formatted : 378
Sentences formatted : 379
Sentences formatted : 380
Sentences formatted : 381
Sentences formatted : 382
Sentences formatted : 383
Sentences formatted : 384
Sentences formatted : 385
Sentences formatted : 386
Sentences formatted : 387
Sentences formatted : 388
Sentences formatted : 389
Sentences formatted : 390
Sentences formatted : 391
Sentences formatted : 392
Sentences formatted : 393
Sentences formatted : 394
Sentences formatted : 395
Sentences formatted : 396
Sentences formatted : 397
Sentences formatted : 398
Sentences formatted : 399
Sentences formatted : 400
Sentences formatted : 401
Sentences formatted : 402
Sentences formatted : 403
Sentences formatted : 404
Sentences formatted : 405
Sentences fo

Sentences formatted : 792
Sentences formatted : 793
Sentences formatted : 794
Sentences formatted : 795
Sentences formatted : 796
Sentences formatted : 797
Sentences formatted : 798
Sentences formatted : 799
Sentences formatted : 800
Sentences formatted : 801
Sentences formatted : 802
Sentences formatted : 803
Sentences formatted : 804
Sentences formatted : 805
Sentences formatted : 806
Sentences formatted : 807
Sentences formatted : 808
Sentences formatted : 809
Sentences formatted : 810
Sentences formatted : 811
Sentences formatted : 812
Sentences formatted : 813
Sentences formatted : 814
Sentences formatted : 815
Sentences formatted : 816
Sentences formatted : 817
Sentences formatted : 818
Sentences formatted : 819
Sentences formatted : 820
Sentences formatted : 821
Sentences formatted : 822
Sentences formatted : 823
Sentences formatted : 824
Sentences formatted : 825
Sentences formatted : 826
Sentences formatted : 827
Sentences formatted : 828
Sentences formatted : 829
Sentences fo

Sentences formatted : 1125
Sentences formatted : 1126
Sentences formatted : 1127
Sentences formatted : 1128
Sentences formatted : 1129
Sentences formatted : 1130
Sentences formatted : 1131
Sentences formatted : 1132
Sentences formatted : 1133
Sentences formatted : 1134
Sentences formatted : 1135
Sentences formatted : 1136
Sentences formatted : 1137
Sentences formatted : 1138
Sentences formatted : 1139
Sentences formatted : 1140
Sentences formatted : 1141
Sentences formatted : 1142
Sentences formatted : 1143
Sentences formatted : 1144
Sentences formatted : 1145
Sentences formatted : 1146
Sentences formatted : 1147
Sentences formatted : 1148
Sentences formatted : 1149
Sentences formatted : 1150
Sentences formatted : 1151
Sentences formatted : 1152
Sentences formatted : 1153
Sentences formatted : 1154
Sentences formatted : 1155
Sentences formatted : 1156
Sentences formatted : 1157
Sentences formatted : 1158
Sentences formatted : 1159
Sentences formatted : 1160
Sentences formatted : 1161
S

Sentences formatted : 1533
Sentences formatted : 1534
Sentences formatted : 1535
Sentences formatted : 1536
Sentences formatted : 1537
Sentences formatted : 1538
Sentences formatted : 1539
Sentences formatted : 1540
Sentences formatted : 1541
Sentences formatted : 1542
Sentences formatted : 1543
Sentences formatted : 1544
Sentences formatted : 1545
Sentences formatted : 1546
Sentences formatted : 1547
Sentences formatted : 1548
Sentences formatted : 1549
Sentences formatted : 1550
Sentences formatted : 1551
Sentences formatted : 1552
Sentences formatted : 1553
Sentences formatted : 1554
Sentences formatted : 1555
Sentences formatted : 1556
Sentences formatted : 1557
Sentences formatted : 1558
Sentences formatted : 1559
Sentences formatted : 1560
Sentences formatted : 1561
Sentences formatted : 1562
Sentences formatted : 1563
Sentences formatted : 1564
Sentences formatted : 1565
Sentences formatted : 1566
Sentences formatted : 1567
Sentences formatted : 1568
Sentences formatted : 1569
S

...

Sentences formatted : 46418
Sentences formatted : 46419
Sentences formatted : 46420
Sentences formatted : 46421
Sentences formatted : 46422
Sentences formatted : 46423
Sentences formatted : 46424
Sentences formatted : 46425
Sentences formatted : 46426
Sentences formatted : 46427
Sentences formatted : 46428
Sentences formatted : 46429
Sentences formatted : 46430
Sentences formatted : 46431
Sentences formatted : 46432
Sentences formatted : 46433
Sentences formatted : 46434
Sentences formatted : 46435
Sentences formatted : 46436
Sentences formatted : 46437
Sentences formatted : 46438
Sentences formatted : 46439
Sentences formatted : 46440
Sentences formatted : 46441
Sentences formatted : 46442
Sentences formatted : 46443
Sentences formatted : 46444
Sentences formatted : 46445
Sentences formatted : 46446
Sentences formatted : 46447
Sentences formatted : 46448
Sentences formatted : 46449
Sentences formatted : 46450
Sentences formatted : 46451
Sentences formatted : 46452
Sentences formatted 

Sentences formatted : 46715
Sentences formatted : 46716
Sentences formatted : 46717
Sentences formatted : 46718
Sentences formatted : 46719
Sentences formatted : 46720
Sentences formatted : 46721
Sentences formatted : 46722
Sentences formatted : 46723
Sentences formatted : 46724
Sentences formatted : 46725
Sentences formatted : 46726
Sentences formatted : 46727
Sentences formatted : 46728
Sentences formatted : 46729
Sentences formatted : 46730
Sentences formatted : 46731
Sentences formatted : 46732
Sentences formatted : 46733
Sentences formatted : 46734
Sentences formatted : 46735
Sentences formatted : 46736
Sentences formatted : 46737
Sentences formatted : 46738
Sentences formatted : 46739
Sentences formatted : 46740
Sentences formatted : 46741
Sentences formatted : 46742
Sentences formatted : 46743
Sentences formatted : 46744
Sentences formatted : 46745
Sentences formatted : 46746
Sentences formatted : 46747
Sentences formatted : 46748
Sentences formatted : 46749
Sentences formatted 

Sentences formatted : 47022
Sentences formatted : 47023
Sentences formatted : 47024
Sentences formatted : 47025
Sentences formatted : 47026
Sentences formatted : 47027
Sentences formatted : 47028
Sentences formatted : 47029
Sentences formatted : 47030
Sentences formatted : 47031
Sentences formatted : 47032
Sentences formatted : 47033
Sentences formatted : 47034
Sentences formatted : 47035
Sentences formatted : 47036
Sentences formatted : 47037
Sentences formatted : 47038
Sentences formatted : 47039
Sentences formatted : 47040
Sentences formatted : 47041
Sentences formatted : 47042
Sentences formatted : 47043
Sentences formatted : 47044
Sentences formatted : 47045
Sentences formatted : 47046
Sentences formatted : 47047
Sentences formatted : 47048
Sentences formatted : 47049
Sentences formatted : 47050
Sentences formatted : 47051
Sentences formatted : 47052
Sentences formatted : 47053
Sentences formatted : 47054
Sentences formatted : 47055
Sentences formatted : 47056
Sentences formatted 

Sentences formatted : 47318
Sentences formatted : 47319
Sentences formatted : 47320
Sentences formatted : 47321
Sentences formatted : 47322
Sentences formatted : 47323
Sentences formatted : 47324
Sentences formatted : 47325
Sentences formatted : 47326
Sentences formatted : 47327
Sentences formatted : 47328
Sentences formatted : 47329
Sentences formatted : 47330
Sentences formatted : 47331
Sentences formatted : 47332
Sentences formatted : 47333
Sentences formatted : 47334
Sentences formatted : 47335
Sentences formatted : 47336
Sentences formatted : 47337
Sentences formatted : 47338
Sentences formatted : 47339
Sentences formatted : 47340
Sentences formatted : 47341
Sentences formatted : 47342
Sentences formatted : 47343
Sentences formatted : 47344
Sentences formatted : 47345
Sentences formatted : 47346
Sentences formatted : 47347
Sentences formatted : 47348
Sentences formatted : 47349
Sentences formatted : 47350
Sentences formatted : 47351
Sentences formatted : 47352
Sentences formatted 

Sentences formatted : 47613
Sentences formatted : 47614
Sentences formatted : 47615
Sentences formatted : 47616
Sentences formatted : 47617
Sentences formatted : 47618
Sentences formatted : 47619
Sentences formatted : 47620
Sentences formatted : 47621
Sentences formatted : 47622
Sentences formatted : 47623
Sentences formatted : 47624
Sentences formatted : 47625
Sentences formatted : 47626
Sentences formatted : 47627
Sentences formatted : 47628
Sentences formatted : 47629
Sentences formatted : 47630
Sentences formatted : 47631
Sentences formatted : 47632
Sentences formatted : 47633
Sentences formatted : 47634
Sentences formatted : 47635
Sentences formatted : 47636
Sentences formatted : 47637
Sentences formatted : 47638
Sentences formatted : 47639
Sentences formatted : 47640
Sentences formatted : 47641
Sentences formatted : 47642
Sentences formatted : 47643
Sentences formatted : 47644
Sentences formatted : 47645
Sentences formatted : 47646
Sentences formatted : 47647
Sentences formatted 

In [ ]:
print(len(x_data))

Okay, so we have processed 47761 sentences. Let's verify their format

In [17]:
print(x_data[1213])

[513, 122, 123, 9, 3659, 58, 4909, 11, 9, 1556, 90, 599, 90, 34, 841, 34, 4264, 13, 4910, 21]


In [19]:
for i in y_data[1213]:
    print(i)

[0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
[0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]


So, for every word, we have a class it belongs to.

### 3. Making the model

In [20]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM

Using TensorFlow backend.
